In [1]:
import numpy as np 
import pandas as pd 

df=pd.DataFrame(columns=['path','label'])

import os
for dirname, _, filenames in os.walk('../input/fire-dataset/fire_dataset/fire_images'):
    for filename in filenames:
        df = df.append(pd.DataFrame([[os.path.join(dirname, filename),'fire']],columns=['path','label']))
        
for dirname, _, filenames in os.walk('../input/fire-dataset/fire_dataset/non_fire_images'):
    for filename in filenames:
        df = df.append(pd.DataFrame([[os.path.join(dirname, filename), 'non-fire']], columns=['path','label']))

In [3]:
df=df.sample(frac=1).reset_index(drop=True)
df.head()

In [7]:
df['label'].value_counts().plot(kind='bar')

In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
generator = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=2,
        zoom_range=0.2,
        rescale=1/255,
        validation_split=0.2,
)

In [16]:
train_gen = generator.flow_from_dataframe(df,x_col='path',y_col='label', image_sizes=(256,256),class_mode='binary', subset='training')
valid_gen = generator.flow_from_dataframe(df,x_col='path', y_col='label', image_sizes=(256,256), class_mode='binary', subset='validation')

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

In [19]:
model=Sequential()
model.add(Conv2D(filters=32, kernel_size=(2,2), activation='relu', input_shape=(256,256,3)))
model.add(MaxPool2D())
model.add(Conv2D(filters=64, kernel_size=(2,2), activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(filters=128, kernel_size=(2,2), activation='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [20]:
model.summary()

In [32]:
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

In [33]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=5)

In [34]:
model.fit(x=train_gen, batch_size=32, epochs=15, validation_data = valid_gen, callbacks=[early_stopping,reduce_lr_on_plateau])